In [1]:
import numpy as np
import pandas as pd

In [2]:
path = "C:/Users/User/Desktop/AI/10. Artificial Intelligence/4. 4rth semester/Advanced ML/project/datasets/DeliciousMIL/Data/train-data.dat"

In [3]:
labels = []

for item in open("C:/Users/User/Desktop/AI/10. Artificial Intelligence/4. 4rth semester/Advanced ML/project/datasets/DeliciousMIL/Data/labels.txt").readlines():
    item = item.replace('\n', '')
    label_name, _ = item.split(', ')
    labels.append(label_name)

In [4]:
labels

['programming',
 'style',
 'reference',
 'java',
 'web',
 'internet',
 'culture',
 'design',
 'education',
 'language',
 'books',
 'writing',
 'computer',
 'english',
 'politics',
 'history',
 'philosophy',
 'science',
 'religion',
 'grammar']

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer

# path = "./Data"
def read_train(path):
    with open(path) as f:
        line = f.readline()
        docs = []
        docs_bin = []
        while line:
            line = line.rstrip('\n').split()
            L = []
            for i in line:
                try:
                    L.append(int(i))
                except:
                    continue
            docs.append(np.array(L))

            words_bin = np.zeros(8520)
            for i in L:
                words_bin[i] += 1

            docs_bin.append(words_bin)
            line = f.readline()
        docs = np.array(docs)
        docs_bin = np.array(docs_bin)
    return docs_bin, docs

# normalize
def tfIdf(docs_bin):
    tf_transformer = TfidfTransformer(use_idf=False).fit(docs_bin)
    X_train_tf = tf_transformer.transform(docs_bin)
    return X_train_tf

In [6]:
X_train1,train = read_train(path)
X_train = tfIdf(X_train1)

In [7]:
print(X_train.shape)

(8251, 8520)


In [8]:
path2 = "C:/Users/User/Desktop/AI/10. Artificial Intelligence/4. 4rth semester/Advanced ML/project/datasets/DeliciousMIL/Data/test-data.dat"

In [9]:
X_test1, test = read_train(path2)
X_test = tfIdf(X_test1)

In [10]:
y_train = np.loadtxt("C:/Users/User/Desktop/AI/10. Artificial Intelligence/4. 4rth semester/Advanced ML/project/datasets/DeliciousMIL/Data/train-label.dat")
y_test =  np.loadtxt("C:/Users/User/Desktop/AI/10. Artificial Intelligence/4. 4rth semester/Advanced ML/project/datasets/DeliciousMIL/Data/test-label.dat")

In [11]:
print("X_train shape: ",X_train.shape)
print("y_train shape: ",y_train.shape)
print("X_test shape: ",X_test.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (8251, 8520)
y_train shape:  (8251, 20)
X_test shape:  (3983, 8520)
y_test shape:  (3983, 20)


 #  Train 

In [18]:
# using Label Powerset
# train the TFId X_train
import time
from skmultilearn.problem_transform import LabelPowerset
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

classifiers = [RandomForestClassifier(n_estimators=50, criterion='gini'),
               SVC(kernel='rbf', C=1),
              KNeighborsClassifier(n_neighbors=60)]
state=[]
for classifier in classifiers:
    start = time.time()
    clf = LabelPowerset(classifier = classifier, require_dense=[False, True])
    clf.fit(X_train,y_train)
    predictions = clf.predict(X_test)
    print("time: ",  time.time() - start)
    conf_matr = metrics.multilabel_confusion_matrix(y_test, predictions)
    state.append([classifier, conf_matr])

time:  264.2511441707611
time:  804.9882624149323
time:  3.258288860321045


In [19]:
# return the name of the classifier
def print_estimator_name(estimator):
    return estimator.__class__.__name__
def results(state):
    results=[]
    for states in state:
        cl = print_estimator_name(states[0]) #['classifier']
        
        print("\nClassifier: %s " %cl)
        print("Confusion matrices per label \n")
        BA = []
        for i in range(len(labels)):
            print("Label: ", labels[i])
            print(states[-1][i])
            TPR = states[-1][i][1][1]/(states[-1][i][1][1]+states[-1][i][1][0]) # TPR=TP/(TP+FN) #Recall
            TNR = states[-1][i][0][0]/(states[-1][i][0][0]+states[-1][i][0][1]) # TNR = TN/(TN+FP) #specificity
            balanced_accuracy = round((TPR+TNR)/2,3)
#             print("Balanced_accuracy: ", balanced_accuracy)
            BA.append(balanced_accuracy)
            results.append([cl, labels[i], balanced_accuracy])
        print("\nBalanced accuracy per label ")
        for i in range(len(labels)):
            print(str(labels[i]) + " : %s " % (BA[i]))
    return results, BA

In [20]:
# acquire the results
result = results(state)


Classifier: RandomForestClassifier 
Confusion matrices per label 

Label:  programming
[[2706  300]
 [ 495  482]]
Label:  style
[[3542  213]
 [ 132   96]]
Label:  reference
[[2098  327]
 [1188  370]]
Label:  java
[[3417  194]
 [ 143  229]]
Label:  web
[[2606  327]
 [ 721  329]]
Label:  internet
[[3350   96]
 [ 498   39]]
Label:  culture
[[3190   91]
 [ 651   51]]
Label:  design
[[2547  357]
 [ 726  353]]
Label:  education
[[3040  140]
 [ 649  154]]
Label:  language
[[3441   59]
 [ 407   76]]
Label:  books
[[3417   59]
 [ 419   88]]
Label:  writing
[[3450   55]
 [ 425   53]]
Label:  computer
[[3387   87]
 [ 472   37]]
Label:  english
[[3575   53]
 [ 296   59]]
Label:  politics
[[3471  120]
 [ 250  142]]
Label:  history
[[3485   57]
 [ 414   27]]
Label:  philosophy
[[3696   18]
 [ 250   19]]
Label:  science
[[3405   77]
 [ 413   88]]
Label:  religion
[[3743   33]
 [ 139   68]]
Label:  grammar
[[3817   33]
 [ 108   25]]

Balanced accuracy per label 
programming : 0.697 
style : 0.682 
re